In [1]:
from math import exp
import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [2]:

zeta, tau, gamma =0.00996293, 0.19959996, 4.62105268



In [3]:
def get_fractional_series(X0):
    """
    :param X0: 待计算分数阶的序列
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr

In [4]:
data = pd.read_excel("使用版2012-2023-2个相关因素_2位小数.xlsx")
n = data.shape[0]
train_n = n-2

In [5]:
# 第一列是年份，故先舍弃
X0 = data.iloc[:train_n, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T
y1 = X1[:, 0] ** (1 - gamma)

Zy = (y1[1:] + y1[:-1]) / 2
M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

In [6]:
# 构建 B
TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
ONE = np.ones(train_n - 1)
B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

# 构建 Y
Y = np.diff(y1)

# 最小二乘法
a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
a, b2, b3, h1, h2

(-0.1490709727815055,
 -0.00023690436345393098,
 9.102593308975537e-05,
 2.3006045407652305e-05,
 0.0002177034461270037)

In [7]:
# 求 mu
mu1 = (1-gamma)/(1+0.5*a*(1-gamma))
mu2 = (1-0.5*a*(1-gamma))/(1+0.5*a*(1-gamma))
mu3 = (h1*(1-gamma)*(exp(tau)-1))/(1+0.5*a*(1-gamma))/tau
mu4 = (h2*(1-gamma))/(1+0.5*a*(1-gamma))
mu1,mu2,mu3,mu4

(-2.851454015777545,
 0.5749309760263112,
 -7.260585061631792e-05,
 -0.0006207713657074551)

In [8]:
# 这里重新获取下原数据，以实现所有数据的预测
# 第一列是年份，故先舍弃
X0 = data.iloc[:, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T

# 求 hat y1(1为上标的那个haty^1)
hat_y1 = [y1[0]]
for m in range(2, n + 1):
    hat_ym = 0
    hat_ym += mu2 ** (m - 1) * y1[0]
    for v in range(1, m):
        hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
    for w in range(0, m - 1):
        hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
    hat_y1.append(hat_ym)
print(hat_y1)

[0.015301348041554258, 0.00935856451034354, 0.005930651728580612, 0.003854351820081231, 0.002495383827563398, 0.0018149376098969805, 0.0015198973904085513, 0.0011988707689730225, 0.0009966213860334772, 0.0006335132962105183, 0.00030570133217303394, 0.0003024249514731193]


In [10]:
hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))
print(hat_x1)
hat_x0 = []
for k in range(2, n + 1):
    hat_x0_m = 0
    for j in range(0, k):
        hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
    hat_x0.append(hat_x0_m)
print(hat_x0)

[3.17183149 3.63309297 4.12084464 4.64162876 5.23375998 5.71479153
 6.00175172 6.408175   6.74363824 7.6425106  9.3460927  9.37394599]
[3.6014922305258055, 4.069005441561103, 4.5722785713507195, 5.147550058115695, 5.61120025535268, 5.881503016434282, 6.273378533378868, 6.594057971716738, 7.478813486945153, 9.162970442667092, 9.160769318869795]


In [11]:
original_x1 = data.iloc[1:, 1]
APE = np.abs(hat_x0 - original_x1) / original_x1
MAPE=np.mean(APE)
print(APE)
print(MAPE)

1     0.004398
2     0.002212
3     0.028076
4     0.018524
5     0.004688
6     0.007396
7     0.043260
8     0.002903
9     0.063871
10    0.023083
11    0.003623
Name: 主营业务收入（亿元）, dtype: float64
0.018366832848799215


In [24]:
pd.DataFrame(hat_x0).to_excel('FNGBM_hatx0_case1-7.xlsx')
pd.DataFrame(APE).to_excel('FNGBM_APE-7.xlsx')

In [112]:
original_x1

4      28465.4700
5      35984.4000
6      43206.3000
7      52799.1000
8      59688.8000
9      67584.2000
10     78309.9325
11     87304.6806
12     92969.5665
13     98634.4524
14    100098.0000
15    110085.9746
16    132988.2635
17    149087.7480
Name: 主营业务收入（亿元）, dtype: float64